# CNN multi-class image classifier

The example dataset is the CIFAR-10 dataset, which consists of 60000 small color images (32x32) in 10 classes with 6000 example each. The 10 classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.
More information about the dataset can be found: 
https://www.cs.toronto.edu/~kriz/cifar.html

Here, CNN is implemented using the Keras library. 

# Import libraries and modules 

In [101]:
from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
#Core layers
from keras.layers import Dense, Dropout, Activation, Flatten
#CNN layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import SeparableConv2D

import numpy as np


In [102]:
#Setting up some parameters for the classifier to use later. 
batch_size = 64
num_classes = 10
epochs = 30


# Data preparation

In [103]:
#Shuffle and split the dataset into train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Usually 2-D image data is stored as (m * pixel * pixel * RGB), 
#here m is the total number of examples
print(x_train.shape)
print(x_test.shape)

#Assure the data is casted to the right datatype. 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Feature normalization 
x_train /= 255
x_test /= 255

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In RGB, the maximum value is 255. Usually normalization should be divided by mean, but in practice 255 seems working just fine. Andrew Ng also mentioned this in his Deep Learning course at Cousera. 

In [104]:
#Now check out how the images are labeled
print(y_train.shape)
print(y_test.shape)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(50000, 1)
(10000, 1)


# Training the classifer 

In [105]:
#Declare a sequential model
model = Sequential()
#CNN input layer 
model.add(SeparableConv2D(32, kernel_size =(3,3), 
                 activation='relu', 
                 depth_multiplier = 3,
                 padding = 'same',
                 input_shape=x_train.shape[1:]))

#Add hidden layers to the model 
model.add(Conv2D(32,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Fully connected Dense layers 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_2 (Separabl (None, 32, 32, 32)        401       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 6, 6, 64)          0         
__________

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [ ]:
#Compile model
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
#Training
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 133s 3ms/step - loss: 1.7100 - acc: 0.3640 - val_loss: 1.3855 - val_acc: 0.5041
Epoch 2/30
50000/50000 [==============================] - 132s 3ms/step - loss: 1.3512 - acc: 0.5115 - val_loss: 1.1948 - val_acc: 0.5659
Epoch 3/30
50000/50000 [==============================] - 135s 3ms/step - loss: 1.2114 - acc: 0.5671 - val_loss: 1.0491 - val_acc: 0.6348
Epoch 4/30
50000/50000 [==============================] - 138s 3ms/step - loss: 1.1046 - acc: 0.6075 - val_loss: 0.9795 - val_acc: 0.6614
Epoch 5/30
50000/50000 [==============================] - 137s 3ms/step - loss: 1.0461 - acc: 0.6307 - val_loss: 0.8967 - val_acc: 0.6902
Epoch 6/30
50000/50000 [==============================] - 138s 3ms/step - loss: 0.9852 - acc: 0.6534 - val_loss: 0.8897 - val_acc: 0.6978
Epoch 7/30
50000/50000 [==============================] - 134s 3ms/step - loss: 0.9415 - acc: 0.6687 - val_loss: 0.8214 - 